In [3]:
pip install pyowm

Note: you may need to restart the kernel to use updated packages.


In [16]:
from pyowm import OWM
from pyowm.utils import config
from pyowm.utils import timestamps

owm = OWM('8a52d544764a1ad5395f20f9a42f4391')
mgr = owm.weather_manager()

In [17]:
# current weather in Berlin
observation = mgr.weather_at_place('Berlin,Ger')
w = observation.weather

In [19]:
w.status

'Drizzle'

In [18]:
w.detailed_status

'light intensity drizzle'

In [20]:
w.wind()

{'speed': 3.58, 'deg': 250, 'gust': 5.81}

In [21]:
w.humidity

91

In [22]:
w.temperature('celsius')

{'temp': 6.88,
 'temp_max': 7.83,
 'temp_min': 6.15,
 'feels_like': 4.4,
 'temp_kf': None}

In [23]:
w.rain

{}

In [24]:
w.heat_index

In [25]:
w.clouds

75

In [26]:
w.pressure

{'press': 1017, 'sea_level': None}

In [30]:
# empty sunrise_time() than unix time format, use 'iso' or 'date'
w.sunrise_time(timeformat='iso')

'2021-11-23 06:42:16+00:00'

In [31]:
w.sunset_time(timeformat='iso')

'2021-11-23 15:03:40+00:00'

In [57]:
# request one-call api - get object that contains all weather info
one_call = mgr.one_call(lat=52.5244, lon=13.4105, exclude='minutely', units='metric')
one_call.forecast_hourly

[<pyowm.weatherapi25.weather.Weather - reference_time=2021-11-23 15:00:00+00:00, status=clouds, detailed_status=broken clouds>,
 <pyowm.weatherapi25.weather.Weather - reference_time=2021-11-23 16:00:00+00:00, status=clouds, detailed_status=broken clouds>,
 <pyowm.weatherapi25.weather.Weather - reference_time=2021-11-23 17:00:00+00:00, status=clouds, detailed_status=broken clouds>,
 <pyowm.weatherapi25.weather.Weather - reference_time=2021-11-23 18:00:00+00:00, status=clouds, detailed_status=broken clouds>,
 <pyowm.weatherapi25.weather.Weather - reference_time=2021-11-23 19:00:00+00:00, status=clouds, detailed_status=overcast clouds>,
 <pyowm.weatherapi25.weather.Weather - reference_time=2021-11-23 20:00:00+00:00, status=clouds, detailed_status=overcast clouds>,
 <pyowm.weatherapi25.weather.Weather - reference_time=2021-11-23 21:00:00+00:00, status=clouds, detailed_status=overcast clouds>,
 <pyowm.weatherapi25.weather.Weather - reference_time=2021-11-23 22:00:00+00:00, status=clouds, de

In [55]:
one_call.current.temp

{'temp': 6.9, 'feels_like': 4.7}

In [60]:
one_call

<pyowm.weatherapi25.one_call.OneCall - lat=52.5244, lon=13.4105, retrieval_time=1637682760>

In [64]:
from pyowm import OWM
from pyowm.utils import config
from pyowm.utils import timestamps
import pandas as pd

OWM_key = "8a52d544764a1ad5395f20f9a42f4391"
city = "Berlin"
country = "DE"

owm = OWM(OWM_key)
mgr = owm.weather_manager()

# Will it be clear tomorrow at this time in Milan (Italy) ?
forecast = mgr.forecast_at_place(city + ',' + country, '3h')
# answer = forecast.will_be_clear_at(timestamps.tomorrow())

from numpy import nan

def forecast_extraction(forecast_object): 
    """
    Given a forecast object from Open weathor Pyhton API, 
    return a structured dataframe. 

    :param: forecast object
    :returns: a dataframe with specific weather information
    """
    forecast_df = pd.json_normalize({
        'time' : forecast_object.reference_time(timeformat='date'), 
        'temperature' : forecast_object.temperature(unit='celsius'), 
        'precipitation_probabily' : forecast_object.precipitation_probability, 
        'rain' : forecast_object.rain, 
        'humidity' : forecast_object.humidity, 
        'status' : forecast_object.status, 
        'snow' : [nan if forecast_object.snow is None else forecast_object.snow], 
        'wind' : forecast_object.wind('km_hour')
    })

    return(forecast_df)

weather_forecast = pd.concat(
    [forecast_extraction(weather) for weather in forecast.forecast.weathers]
)

weather_forecast.head()

,time,precipitation_probabily,humidity,status,snow,temperature.temp,temperature.temp_kf,temperature.temp_max,temperature.temp_min,temperature.feels_like,wind.speed,wind.deg,wind.gust
0,2021-11-24 12:00:00+00:00,0.0,90,Clouds,[{}],7.58,-272.71,7.58,7.14,6.26,7.668,241,17.604
0,2021-11-24 15:00:00+00:00,0.0,85,Clouds,[{}],7.37,-272.74,7.37,6.96,6.20,6.912,210,13.356
0,2021-11-24 18:00:00+00:00,0.0,78,Clouds,[{}],7.10,-272.91,7.10,6.86,5.91,6.840,195,12.528
0,2021-11-24 21:00:00+00:00,0.0,73,Clouds,[{}],6.15,-273.15,6.15,6.15,4.71,7.200,176,15.768
0,2021-11-25 00:00:00+00:00,0.0,78,Clouds,[{}],4.16,-273.15,4.16,4.16,1.88,9.108,156,24.804
